# Lab3.4 Sentiment and Emotion Classification using transformer models

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

This notebook explains how you can use a transformer model that is fine-tuned for sentiment analysis. Fine-tuned transformer models are published regularly on the huggingface platform: https://huggingface.co/models

These models are very big (Gigabytes) and require a computer with sufficient memory to load. Furthermore, loading these models takes some time as well. It is also possible to copy such a model to your disk and to load the local copy. Still a substantial memory is needed to load it.

## NLP pipelines for transformers

Huggingface transfomers provides an option to create an **pipeline** to perform a NLP task with a pretrained model: 

"The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering."

More information can be found here: 

https://huggingface.co/docs/transformers/main_classes/pipelines

The pipelines are abstractions from specific task such as sentiment-analysis and entity recognition. In the case of sentiment-analysis, the complete sentence representation of the model is taken as the input and classified for the the defined labels. In the case of entity recognition, each token in a sentence is classified separately in a sequence, i.e. a sequence classification task. Whereas a finetuned model can be used for a task depends on the way it was fine tuned with labeled data. 

We will use the pipeline module to load a fine-tuned model to perform sentiment analysis

In [1]:
from transformers import pipeline

We load a transformer model 'distilbert-base-uncased-finetuned-sst-2-english' that is fine-tuned for binary classification from the Hugging face repository:

https://huggingface.co/models

We need to load the model for the sequence classifcation and the tokenizer to convert the sentences into tokens according to the vocabulary of the model.

Loading the model takes some time.

## Using an English fine-tuned transformer model

In [4]:
sentimentenglish = pipeline("sentiment-analysis", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

We now created an instantiation of a pipeline that can tokenize any sentence, obtain a sententence embedding from the transformer language model and perform the **sentiment-analysis** task. Let's try it out on an example sentence.

In [5]:
sentence_pos_en = "Nice hotel and the service is great"

In [6]:
sentimentenglish(sentence_pos_en)

[{'label': 'POSITIVE', 'score': 0.999881386756897}]

In [11]:
sentence_neg_en = "The rooms are dirty and the wifi does not work"

In [8]:
sentimentenglish(sentence_neg_en)

[{'label': 'NEGATIVE', 'score': 0.9997870326042175}]

This is easy and seems to work very well. 

You may wonder what happens if you choose a pipeline that does not fit the fine-tuned model? Good question. Let's try this and load the same model for **ner**, which stands for named-entity-recognition and this is a token sequence classification task.

In [9]:
errorpipeline = pipeline("ner", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertForTokenClassification: ['pre_classifier.bias', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We do get an error message that something is wrong but let's see what happens if we give it one of the above sentences:

In [12]:
errorpipeline(sentence_neg_en)

[{'entity': 'POSITIVE',
  'score': 0.5202873,
  'index': 1,
  'word': 'the',
  'start': 0,
  'end': 3},
 {'entity': 'NEGATIVE',
  'score': 0.51403594,
  'index': 2,
  'word': 'rooms',
  'start': 4,
  'end': 9},
 {'entity': 'POSITIVE',
  'score': 0.56954414,
  'index': 3,
  'word': 'are',
  'start': 10,
  'end': 13},
 {'entity': 'POSITIVE',
  'score': 0.5369258,
  'index': 4,
  'word': 'dirty',
  'start': 14,
  'end': 19},
 {'entity': 'POSITIVE',
  'score': 0.54581994,
  'index': 5,
  'word': 'and',
  'start': 20,
  'end': 23},
 {'entity': 'NEGATIVE',
  'score': 0.5072517,
  'index': 6,
  'word': 'the',
  'start': 24,
  'end': 27},
 {'entity': 'NEGATIVE',
  'score': 0.575071,
  'index': 7,
  'word': 'wi',
  'start': 28,
  'end': 30},
 {'entity': 'NEGATIVE',
  'score': 0.53956544,
  'index': 8,
  'word': '##fi',
  'start': 30,
  'end': 32},
 {'entity': 'POSITIVE',
  'score': 0.53940797,
  'index': 9,
  'word': 'does',
  'start': 33,
  'end': 37},
 {'entity': 'POSITIVE',
  'score': 0.5475

Interesting, it now classifies each token as a text rather than the complete sentence. So realise the model and the API do not know what they do internally.

Text classification and token classification are still somewhat similar and our hacking did not break it. This does not always work:

In [13]:
qapipeline = pipeline("question-answering", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertForQuestionAnswering: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stre

In [14]:
qapipeline(sentence_neg_en)

ValueError: T argument needs to be of type (SquadExample, dict)

Apparently, a model trained for Q&A has some other data that is not present.

What does not work either is to give it a pipeline that does not exist:

In [15]:
bullshitpipeline = pipeline("bull-shit", 
                            model="distilbert-base-uncased-finetuned-sst-2-english", 
                            tokenizer="distilbert-base-uncased-finetuned-sst-2-english")

KeyError: "Unknown task bull-shit, available tasks are ['audio-classification', 'automatic-speech-recognition', 'feature-extraction', 'text-classification', 'token-classification', 'question-answering', 'table-question-answering', 'fill-mask', 'summarization', 'translation', 'text2text-generation', 'text-generation', 'zero-shot-classification', 'conversational', 'image-classification', 'image-segmentation', 'object-detection', 'translation_XX_to_YY']"

Nice outcome is that you get a list of all the pipeline names that are supported.

## Using a Dutch fine-tuned transformer model

We can use a fine-tuned Dutch model for Dutch sentiment analysis by creating another pipeline. Again loading this model takes some time. Also note that after loading, both models are loaded in memory. So if you have issues loading, you may want to start over and try again just with the Dutch pipeline.

In [9]:
sentimentdutch = pipeline("sentiment-analysis", 
                          model="wietsedv/bert-base-dutch-cased-finetuned-sentiment", 
                          tokenizer="wietsedv/bert-base-dutch-cased-finetuned-sentiment")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

We test it on two similar Dutch sentences:

In [10]:
sentence_pos_nl="Mooi hotel en de service is geweldig"
sentence_neg_nl="De kamers zijn smerig en de wifi doet het niet"

In [11]:
sentimentdutch(sentence_pos_nl)

[{'label': 'pos', 'score': 0.9999955892562866}]

In [12]:
sentimentdutch(sentence_neg_nl)

[{'label': 'neg', 'score': 0.6675310134887695}]

This seems to work fine too although the score for negative in the second example is much lower.

##  BERT Finetuned for emotion detection with GO dataset

We will now load the language model BERT that is finetuned for emotion detection using the *go_emotions* data set. Go_emotions has 28 nuanced emotion labels including neutral, so many more than the six basic Ekman emotions that we have seen before.

https://github.com/google-research/google-research/tree/master/goemotions

```
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]```

We use define a *sentiment-analysis* pipeline and load the BERT model that was finetuned to classify sentences with the 28 GO_EMOTION labels. Note that the fine-tuning is not on sentiments but on different labels. The pipeline **sentiment-analysis** should be read as a name for that type of classification that is carried out. In this case it is text classification in which is a label is assigned to the text as a whole and not to for example individual tokens. The **sentiment-analysis** pipeline can be used as an alias for using any model that is fine-tuned for text classification, regardless of the labels that have been used.

To put it simply, the pipeline models the text according the the language model and performs the classification task that it is fine-tuned for. It simply returns the labels from fine-tuning, which can be sentiment, emotion, topics, or anything else that is assocaited with the text as a whole.

Furthermore, we can use a parameter to either get the highest scoring label or all labels with a score.
Below we set the parameter *return_all_scores* to True to get a score over all labels.

In [2]:
model_name = "bhadresh-savani/bert-base-go-emotion" 
emotion_pipeline = pipeline('sentiment-analysis', 
                    model=model_name, return_all_scores=True, truncation=True)

We now created an instance *emotion_pipeline* of a transformer pipeline in analogy of an sentiment analysis classification task that we can apply to any utterance. The pipeline will use the tokenizer of the finetuned model and feed the sentence representation to the classifier as a sequence of contextualized token representations.

In [3]:
emotion_labels = emotion_pipeline("Thanks for using it.")

In [4]:
for result in emotion_labels[0]:
    print(result)

{'label': 'admiration', 'score': 0.0007500764913856983}
{'label': 'amusement', 'score': 0.00011047106818296015}
{'label': 'anger', 'score': 9.69245083979331e-05}
{'label': 'annoyance', 'score': 0.0002597433340270072}
{'label': 'approval', 'score': 0.0011426000855863094}
{'label': 'caring', 'score': 0.00030970710213296115}
{'label': 'confusion', 'score': 0.00014959769032429904}
{'label': 'curiosity', 'score': 0.00015838834224268794}
{'label': 'desire', 'score': 0.0001385686337016523}
{'label': 'disappointment', 'score': 0.00016352151578757912}
{'label': 'disapproval', 'score': 0.00020030527957715094}
{'label': 'disgust', 'score': 5.9684312873287126e-05}
{'label': 'embarrassment', 'score': 5.588319982052781e-05}
{'label': 'excitement', 'score': 0.00018467492191120982}
{'label': 'fear', 'score': 5.239497113507241e-05}
{'label': 'gratitude', 'score': 0.9934592247009277}
{'label': 'grief', 'score': 2.022587250394281e-05}
{'label': 'joy', 'score': 0.0003203642263542861}
{'label': 'love', 'sc

The GO emotions are a lot more nuanced than the Ekman emotions. They are based on a diverse range of emotion data and not just facial expression:

https://ai.googleblog.com/2021/10/goemotions-dataset-for-fine-grained.html

Nevertheless it is possible to map the more specific emotions to Ekman's more basic ones and even to sentiments. The next specifications from the original Github of goemotions just do that:

In [5]:
### Mapping GO_Emotions to sentiment values
sentiment_map={
"positive": ["amusement", "excitement", "joy", "love", "desire", "optimism", "caring", "pride", "admiration", "gratitude", "relief", "approval"],
"negative": ["fear", "nervousness", "remorse", "embarrassment", "disappointment", "sadness", "grief", "disgust", "anger", "annoyance", "disapproval"],
"ambiguous": ["realization", "surprise", "curiosity", "confusion"]
}

### Mapping GO_Emotions to Ekman values
ekman_map={
"anger": ["anger", "annoyance", "disapproval"],
"disgust": ["disgust"],
"fear": ["fear", "nervousness"],
"joy": ["joy", "amusement", "approval", "excitement", "gratitude",  "love", "optimism", "relief", "pride", "admiration", "desire", "caring"],
"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
"surprise": ["surprise", "realization", "confusion", "curiosity"],
"neutral": ["neutral"]
}

We can now make a few simple auxiliary functions that can make the translation from the GO emotions to Ekman or sentiment values. The functions sort the predictions by their score but also select only the scores above a certain threshold and map these to one of the mappings.

In [6]:
### Sort a list of results in JSON format by the value of the score element
def sort_predictions(predictions):
    return sorted(predictions, key=lambda x: x['score'], reverse=True)


### Use a mapping to get a dictionary of the mapped GO_emotion scores above the thereshold
def get_mapped_scores(emotion_map, go_emotion_scores, threshold):
    mapped_scores = {}

    for prediction in go_emotion_scores[0]:
        if prediction['score']>=threshold:
            go_emotion=prediction['label']
            for key in emotion_map:
                if go_emotion in emotion_map[key]:
                    if not key in mapped_scores:
                        mapped_scores[key]= [prediction['score']]
                    else:
                        mapped_scores[key].append(prediction['score'])
    return mapped_scores

### Get the averaged score for an emotion or sentiment from the GO_emotion scores above a threshold
### mapped according to the emotion_map
def get_averaged_mapped_scores_by_threshold(emotion_map, go_emotion_scores, threshold):
    averaged_mapped_scores = []
    mapped_scores = get_mapped_scores(emotion_map, go_emotion_scores, threshold)
    for emotion in mapped_scores:
        lst = mapped_scores[emotion]
        averaged_score= sum(lst)/len(lst)
        averaged_mapped_scores.append({'label':emotion, 'score':averaged_score})
    return sort_predictions(averaged_mapped_scores)

Using these function, we can print the averaged Ekman and the averaged sentiment score for any GO emotion classifition result:

In [7]:
threshold = 0.05
ekman_labels = get_averaged_mapped_scores_by_threshold(ekman_map, emotion_labels, threshold)
for ekman in ekman_labels:
    print(ekman)

{'label': 'joy', 'score': 0.9934592247009277}


In [8]:
sentiment_scores = get_averaged_mapped_scores_by_threshold(sentiment_map, emotion_labels, threshold)
for sentiment in sentiment_scores:
    print(sentiment)

{'label': 'positive', 'score': 0.9934592247009277}


# End of this notebook